In [1]:
from typing import List

import numpy as np
import torch
import tensorflow as tf

# 1. Define Check function

In [2]:
def check_einsum(name: str, equation: str, operands: List[np.ndarray]):
    print(f"{'=' * 10} {name} {'=' * 10}")
    print(f"equation: {equation}\n")
    
    print(f"Before {name}:\n{operands}\n")

    numpy_result = np.einsum(equation, *operands)
    torch_result = torch.einsum(equation, *[torch.tensor(operand) for operand in operands])
    tf_result = tf.einsum(equation, *[tf.constant(operand) for operand in operands])
    
    assert np.allclose(numpy_result, torch_result.numpy()), "Numpy result is different from Torch result"
    assert np.allclose(numpy_result, tf_result.numpy()), "Numpy result is different from Tensorflow result"
    
    print("All results are same!!\n")
    print(f"After {name}:\n{numpy_result}\n")
    
    return numpy_result

# 2. Transpose

$$\color{green}{B_{j, i}} \color{black}{=} \color{red}{A_{i,j}}$$

In [3]:
# 2-D Matrix Transpose
test_matrix = np.random.uniform(size=[2,2])
check_einsum("2-D transpose", "ij->ji", [test_matrix])

# 3-D Matrix Transpose
test_matrix = np.random.uniform(size=[2,2,2])
check_einsum("3-D transpose", "ijk->kji", [test_matrix])

========== 2-D transpose ==========
equation: ij->ji

Before 2-D transpose:
[array([[0.3328484 , 0.44398874],
       [0.62238239, 0.56042012]])]

All results are same!!

After 2-D transpose:
[[0.3328484  0.62238239]
 [0.44398874 0.56042012]]

========== 3-D transpose ==========
equation: ijk->kji

Before 3-D transpose:
[array([[[0.00550469, 0.73214113],
        [0.32663122, 0.56243587]],

       [[0.36366162, 0.27567707],
        [0.79475574, 0.55243245]]])]

All results are same!!

After 3-D transpose:
[[[0.00550469 0.36366162]
  [0.32663122 0.79475574]]

 [[0.73214113 0.27567707]
  [0.56243587 0.55243245]]]



array([[[0.00550469, 0.36366162],
        [0.32663122, 0.79475574]],

       [[0.73214113, 0.27567707],
        [0.56243587, 0.55243245]]])

# 3. Sum

$$\color{green}{b} \color{black}{= \sum_{i}\sum_{j}} \color{red}{A_{i,j}} \color{black}{=} \color{red}{A_{i,j}}$$

In [4]:
# 2-D Matrix Sum
test_matrix = np.arange(4).reshape([2,2])
check_einsum("2-D Sum", "ij->", [test_matrix])

# 3-D Matrix Sum
test_matrix = np.arange(8).reshape([2,2,2])
check_einsum("3-D Sum", "ijk->", [test_matrix])

========== 2-D Sum ==========
equation: ij->

Before 2-D Sum:
[array([[0, 1],
       [2, 3]])]

All results are same!!

After 2-D Sum:
6

========== 3-D Sum ==========
equation: ijk->

Before 3-D Sum:
[array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])]

All results are same!!

After 3-D Sum:
28



28

# 4. Column/Row Sum

## Column Sum

$$Column \space sum: \color{green}b_j \color{black}{= \sum_i} \color{red}{A_{i,j}} \color{black}{=} \color{red}{A_{i,j}}$$

## Row Sum

$$Row \space sum: \color{green}b_i \color{black}{= \sum_j} \color{red}{A_{i,j}} \color{black}{=} \color{red}{A_{i,j}}$$

In [5]:
# Column Sum
test_matrix = np.arange(4).reshape([2,2])
check_einsum("2-D Column Sum", "ij->j", [test_matrix])

# Row Sum
check_einsum("2-D Row Sum", "ij->i", [test_matrix])

========== 2-D Column Sum ==========
equation: ij->j

Before 2-D Column Sum:
[array([[0, 1],
       [2, 3]])]

All results are same!!

After 2-D Column Sum:
[2 4]

========== 2-D Row Sum ==========
equation: ij->i

Before 2-D Row Sum:
[array([[0, 1],
       [2, 3]])]

All results are same!!

After 2-D Row Sum:
[1 5]



array([1, 5])

# 5. Matrix-Matrix(Vector) Multiplication

## Matrix-Vector Multiplication

$$\color{green}{c_i} \color{black}{= \sum_j} \color{red}{A_{i,j}} \color{blue}{b_j} \color{black}{=} \color{red}{A_{i,j}} \color{blue}{b_j}$$

## Matrix-Matrix Multiplication

$$\color{green}{C_{i,j}} \color{black}{=\sum_k} \color{red}{A_{i,k}} \color{blue}{B_{k,j}} \color{black}{=}  \color{red}{A_{ik}} \color{blue}{B_{kj}}$$

In [6]:
test_matrix = np.arange(6).reshape([2,3])
test_vector = np.arange(3)
check_einsum("Matrix-Vector Multiplication", "ij,j->i", [test_matrix, test_vector])
check_einsum("Matrix-Matrix Multiplication", "ik,kj->ij", [test_matrix, test_matrix.T])

========== Matrix-Vector Multiplication ==========
equation: ij,j->i

Before Matrix-Vector Multiplication:
[array([[0, 1, 2],
       [3, 4, 5]]), array([0, 1, 2])]

All results are same!!

After Matrix-Vector Multiplication:
[ 5 14]

========== Matrix-Matrix Multiplication ==========
equation: ik,kj->ij

Before Matrix-Matrix Multiplication:
[array([[0, 1, 2],
       [3, 4, 5]]), array([[0, 3],
       [1, 4],
       [2, 5]])]

All results are same!!

After Matrix-Matrix Multiplication:
[[ 5 14]
 [14 50]]



array([[ 5, 14],
       [14, 50]])

# 6. Dot/Outer/Hadamard Product

## Dot Product

$$Dot \space product: \color{green}{c} \color{black}{= \sum_i} \color{red}{A_i} \color{blue}{B_i} \color{black}{=} \color{red}{A_i} \color{blue}{B_i}$$

## Outer Product

$$Outer \space product: \color{green}{C_{i,j}} \color{black}{=} \color{red}{A_i} \color{blue}{B_j} \color{black}{=} \color{red}{A_i} \color{blue}{B_j}$$

## Hadmard Product

$$Hadamard \space product: \color{green}{C_{i,j}} \color{black}{=} \color{red}{A_{i,j}} \color{blue}{B_{i,j}} \color{black}{=} \color{red}{A_{i,j}} \color{blue}{B_{i,j}}$$

In [7]:
test_vector_1 = np.arange(3)
test_vector_2 = np.arange(3)

# Dot Product
check_einsum("Dot product", "i,i->", [test_vector_1, test_vector_2])

# Outer Product
check_einsum("Outer product", "i,j->ij", [test_vector_1, test_vector_2])

test_matrix_1 = np.arange(6).reshape([2,3])
test_matrix_2 = np.arange(6).reshape([2,3])

# Hadamard Product
check_einsum("Hadamard product", "ij,ij->ij", [test_matrix_1, test_matrix_2])

========== Dot product ==========
equation: i,i->

Before Dot product:
[array([0, 1, 2]), array([0, 1, 2])]

All results are same!!

After Dot product:
5

========== Outer product ==========
equation: i,j->ij

Before Outer product:
[array([0, 1, 2]), array([0, 1, 2])]

All results are same!!

After Outer product:
[[0 0 0]
 [0 1 2]
 [0 2 4]]

========== Hadamard product ==========
equation: ij,ij->ij

Before Hadamard product:
[array([[0, 1, 2],
       [3, 4, 5]]), array([[0, 1, 2],
       [3, 4, 5]])]

All results are same!!

After Hadamard product:
[[ 0  1  4]
 [ 9 16 25]]



array([[ 0,  1,  4],
       [ 9, 16, 25]])

# 7. Batch Matrix Multiplication

$$\color{green}{C_{ijl}} \color{black}{=\sum_k} \color{red}{A_{ijk}} \color{blue}{B_{ikl}} \color{black}{=} \color{red}{A_{ijk}} \color{blue}{B_{ikl}}$$

In [8]:
i, j, k, l = 2, 1, 2, 3
test_matrix_1 = np.random.uniform(size=(i,j,k))
test_matrix_2 = np.random.uniform(size=(i,k,l))

einsum_result = check_einsum("Batch Matrix Multiplication", "ijk,ikl->ijl", [test_matrix_1, test_matrix_2])

# coparision between einsum and original bmm

original_bmm = torch.bmm(torch.tensor(test_matrix_1), torch.tensor(test_matrix_2)).numpy()
assert np.all(original_bmm == einsum_result)
print("original bmm result and einsum result are same!")

========== Batch Matrix Multiplication ==========
equation: ijk,ikl->ijl

Before Batch Matrix Multiplication:
[array([[[0.81833742, 0.92045776]],

       [[0.16013933, 0.09363739]]]), array([[[0.69330382, 0.85163496, 0.03238206],
        [0.2413184 , 0.19979804, 0.73615102]],

       [[0.97592846, 0.58599921, 0.95295187],
        [0.35816464, 0.9281158 , 0.15611593]]])]

All results are same!!

After Batch Matrix Multiplication:
[[[0.78947985 0.88083041 0.70409537]]

 [[0.18982213 0.18074786 0.16722336]]]

original bmm result and einsum result are same!


# 8. Bilinear Transformation

$$\color{green}{D_{ij}} \color{black}{=\sum_k\sum_l} \color{red}{A_{ik}} \color{purple}{X_{jkl}} \color{blue}{B_{il}} \color{black}{=} \color{red}{A_{ik}} \color{purple}{X_{jkl}} \color{blue}{B_{il}}$$

In [9]:
i,j,k,l = 2,3,2,2
test_matrix_1 = np.random.uniform(size=(i,k))
test_matrix_2 = np.random.uniform(size=(i,l))
X = np.random.uniform(size=(j,k,l))

einsum_result = check_einsum("Bilinear Transformation", "ik,jkl,il->ij", [test_matrix_1, X, test_matrix_2])

original_bilinear = torch.nn.functional.bilinear(torch.tensor(test_matrix_1), 
                                                 torch.tensor(test_matrix_2), 
                                                 torch.tensor(X)).numpy()
assert np.allclose(original_bilinear, einsum_result)
print("original bilinear result and einsum result are same!")

========== Bilinear Transformation ==========
equation: ik,jkl,il->ij

Before Bilinear Transformation:
[array([[0.21409295, 0.04719953],
       [0.10524751, 0.09642733]]), array([[[0.88813922, 0.13008836],
        [0.23644985, 0.15154198]],

       [[0.90352873, 0.22726196],
        [0.92723979, 0.90313901]],

       [[0.79024212, 0.0382283 ],
        [0.91830727, 0.84841075]]]), array([[0.0901718 , 0.80982622],
       [0.9800735 , 0.11963888]])]

All results are same!!

After Bilinear Transformation:
[[0.04649893 0.09531245 0.05822122]
 [0.11734401 0.19410949 0.17856814]]

original bilinear result and einsum result are same!


# Multi-head Attention(Advanced) 

[Attention is All you Need](https://arxiv.org/abs/1706.03762)

## Multi-head Attention

$$MultiHeadAttention(K,Q,V) = concat(head_1, head_2, head_3, ...)$$

## One head Attention

$$head_i = Attention(QW^Q_i, KW^K_i, VW^V_i)$$

## Self-Attention

$$Attention(Q,K,V) = softmax(\frac{QK^{\top}}{\sqrt{d_k}})V$$

In [10]:
from scipy.special import softmax

def multihead_attention(hidden_states, W_Q, W_K, W_V, num_head):
    batch_size, sequence_length, hidden_size = hidden_states.shape
    assert hidden_size % num_head == 0
    head_hidden_size = hidden_size // num_head

    Q = np.einsum("ijk,kl->ijl", hidden_states, W_Q)   # [batch_size, sequence_length, hidden_size]
    K = np.einsum("ijk,kl->ijl", hidden_states, W_K)   # [batch_size, sequence_length, hidden_size]
    V = np.einsum("ijk,kl->ijl", hidden_states, W_V)   # [batch_size, sequence_length, hidden_size]
    print(f"Q shape: {Q.shape} K shape: {K.shape} V shape: {V.shape}")

    Q = np.reshape(Q, [batch_size, sequence_length, num_head, head_hidden_size]) # [batch_size, sequence_length, num_haed, head_hidden_size]
    K = np.reshape(K, [batch_size, sequence_length, num_head, head_hidden_size]) # [batch_size, sequence_length, num_haed, head_hidden_size]
    V = np.reshape(V, [batch_size, sequence_length, num_head, head_hidden_size]) # [batch_size, sequence_length, num_haed, head_hidden_size]
    print(f"Q shape: {Q.shape} K shape: {K.shape} V shape: {V.shape}")

    Q = np.einsum("ijkl->ikjl", Q)  # [batch_size, num_haed, sequence_length, head_hidden_size]
    K = np.einsum("ijkl->ikjl", K)  # [batch_size, num_haed, sequence_length, head_hidden_size]
    V = np.einsum("ijkl->ikjl", V)  # [batch_size, num_haed, sequence_length, head_hidden_size]
    print(f"Q shape: {Q.shape} K shape: {K.shape} V shape: {V.shape}")

    attention_score = np.einsum("ijkl,ijml->ijkm", Q, K)/np.sqrt(hidden_size)  # [batch_size, num_haed, sequence_length, sequence_length]
    attention_score = softmax(attention_score, axis=3)   # [batch_size, num_haed, sequence_length, sequence_length]
    print(f"Attention score shape: {attention_score.shape}")
    
    attention_result = np.einsum("ijkl,ijkm->iljm", attention_score, V)   # [batch_size, sequence_length, num_head, head_hidden_size]
    attention_result = np.reshape(attention_result, [batch_size, sequence_length, hidden_size])  # [batch_size, sequence_length, hidden_size]
    print(f"Attention result shape: {attention_result.shape}")
    
    return attention_result

In [11]:
batch_size, sequence_length, hidden_size, num_head = 2, 10, 16, 8
hidden_states = np.random.uniform(size=(batch_size, sequence_length, hidden_size))

W_K = np.random.uniform(size=(hidden_size, hidden_size))
W_Q = np.random.uniform(size=(hidden_size, hidden_size))
W_V = np.random.uniform(size=(hidden_size, hidden_size))

result = multihead_attention(hidden_states, W_Q, W_K, W_V, num_head)

Q shape: (2, 10, 16) K shape: (2, 10, 16) V shape: (2, 10, 16)
Q shape: (2, 10, 8, 2) K shape: (2, 10, 8, 2) V shape: (2, 10, 8, 2)
Q shape: (2, 8, 10, 2) K shape: (2, 8, 10, 2) V shape: (2, 8, 10, 2)
Attention score shape: (2, 8, 10, 10)
Attention result shape: (2, 10, 16)
